In [29]:
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# Load data 
indicators_df = pd.read_csv('indicators.csv')
indicators_df.head()

,DIABETE4,_RACE,TOLDHI3,BPHIGH6,_BMI5,SMOKE100,_RFBING5,EDUCA,GENHLTH,_AGEG5YR,EXERANY2,FRUIT2,VEGETAB2,_INCOMG1,MEDCOST1,_SEX
0,No,White,Yes,No,14.54,Yes,Yes,High School Grad,Poor,Age 70 to 74,No,Yes,Yes,"$25,000 to < $35,000",No,Female
1,Yes,Black,No,Yes,28.29,No,Yes,High School Grad,Very Good,Age 70 to 74,No,Yes,Yes,"$15,000 to < $25,000",No,Female
2,Yes,White,Yes,Yes,33.47,No,No,High School Grad,Very Good,Age 60 to 64,Yes,Yes,Yes,"$50,000 to < $100,000",No,Female
3,Yes,Multiracial,Yes,No,28.73,No,Yes,Some High School,Poor,Age 75 to 79,Yes,Yes,Yes,"$15,000 to < $25,000",No,Male
4,No,White,No,No,24.37,Yes,Yes,Some College,Good,Age 80 or older,No,Yes,Yes,"$35,000 to < $50,000",No,Male


In [3]:
indicators_df.dtypes

DIABETE4     object
_RACE        object
TOLDHI3      object
BPHIGH6      object
_BMI5       float64
SMOKE100     object
_RFBING5     object
EDUCA        object
GENHLTH      object
_AGEG5YR     object
EXERANY2     object
FRUIT2       object
VEGETAB2     object
_INCOMG1     object
MEDCOST1     object
_SEX         object
dtype: object

In [4]:
# Generate our categorical variable list
indicators_cat = indicators_df.dtypes[indicators_df.dtypes == "object"].index.tolist()

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(indicators_df[indicators_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(indicators_cat)
encode_df.head()

,DIABETE4_No,DIABETE4_Yes,_RACE_ Native Hawaiian or other Pacific Islander,_RACE_American Indian or Alaskan Native,_RACE_Asian,_RACE_Black,_RACE_Hispanic,_RACE_Multiracial,_RACE_Other,_RACE_White,...,"_INCOMG1_$100,000 to < $200,000","_INCOMG1_$15,000 to < $25,000","_INCOMG1_$25,000 to < $35,000","_INCOMG1_$35,000 to < $50,000","_INCOMG1_$50,000 to < $100,000","_INCOMG1_Less than $15,000",MEDCOST1_No,MEDCOST1_Yes,_SEX_Female,_SEX_Male
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
indicators_df = indicators_df.merge(encode_df,left_index=True, right_index=True)
indicators_df = indicators_df.drop(indicators_cat,1)
indicators_df.head()

C:\Users\ashle\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,_BMI5,DIABETE4_No,DIABETE4_Yes,_RACE_ Native Hawaiian or other Pacific Islander,_RACE_American Indian or Alaskan Native,_RACE_Asian,_RACE_Black,_RACE_Hispanic,_RACE_Multiracial,_RACE_Other,...,"_INCOMG1_$100,000 to < $200,000","_INCOMG1_$15,000 to < $25,000","_INCOMG1_$25,000 to < $35,000","_INCOMG1_$35,000 to < $50,000","_INCOMG1_$50,000 to < $100,000","_INCOMG1_Less than $15,000",MEDCOST1_No,MEDCOST1_Yes,_SEX_Female,_SEX_Male
0,14.54,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,28.29,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,33.47,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,28.73,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,24.37,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [13]:
# Define features and targets
y = indicators_df['DIABETE4_Yes']
X = indicators_df.drop(columns=['DIABETE4_Yes', 'DIABETE4_No'])

In [14]:
# Split into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78, stratify=y)

In [15]:
# Create Standard Scaler
scaler = StandardScaler()

In [16]:
# Fit Data
X_scaler = scaler.fit(X_train)

# Scale Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Create Random Forest Classifier 
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [18]:
# Fit Model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [19]:
# Validate and make predictions with testing data 
predictions = rf_model.predict(X_test_scaled)

In [23]:
# Evaluate performance with accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.826989398390599

In [26]:
# Generate confusion matrix
cm = confusion_matrix(y_test, predictions)

In [27]:
# Create df from confusion matrix
cm_df = pd.DataFrame(cm, index=['Actual 0', 'Acutal 1'], columns = ['Predicted 0', 'Predicted 1'])
cm_df

,Predicted 0,Predicted 1
Actual 0,49744,3257
Acutal 1,7579,2052


In [32]:
# Classification Report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90     53001
         1.0       0.39      0.21      0.27      9631

    accuracy                           0.83     62632
   macro avg       0.63      0.58      0.59     62632
weighted avg       0.79      0.83      0.81     62632



In [33]:
# Rank importance of features
importances = rf_model.feature_importances_
importances

array([0.38430474, 0.00180273, 0.00530045, 0.0042269 , 0.01028391,
       0.0097611 , 0.00495132, 0.00232624, 0.01525455, 0.01197918,
       0.01404168, 0.0239379 , 0.02255112, 0.01717184, 0.01741535,
       0.00539622, 0.00514714, 0.01683704, 0.00434836, 0.01873632,
       0.00047804, 0.01961955, 0.00816816, 0.01103884, 0.01530031,
       0.01023639, 0.00894694, 0.01363283, 0.00218572, 0.00298777,
       0.00456863, 0.00630915, 0.00811558, 0.00984932, 0.01247946,
       0.01430697, 0.01563137, 0.01559725, 0.01422757, 0.01266282,
       0.01115807, 0.00953738, 0.00967601, 0.00559043, 0.0056698 ,
       0.00511174, 0.00512172, 0.00574345, 0.01313163, 0.01391633,
       0.01577637, 0.01688766, 0.01960349, 0.01072371, 0.00800008,
       0.00802679, 0.01712477, 0.01708377])

In [34]:
# Sort features 
sorted(zip(rf_model.feature_importances_,X.columns), reverse=True)

[(0.3843047406263491, '_BMI5'),
 (0.023937901404065386, 'BPHIGH6_No'),
 (0.022551120941494036, 'BPHIGH6_Yes'),
 (0.019619545537496022, 'EDUCA_Some College'),
 (0.019603494218763326, '_INCOMG1_$50,000 to < $100,000'),
 (0.018736322067050056, 'EDUCA_High School Grad'),
 (0.01741535260321007, 'SMOKE100_Yes'),
 (0.017171840256775074, 'SMOKE100_No'),
 (0.017124765241578452, '_SEX_Female'),
 (0.01708377123594685, '_SEX_Male'),
 (0.016887656736197378, '_INCOMG1_$35,000 to < $50,000'),
 (0.016837039360896883, 'EDUCA_College Grad'),
 (0.015776374567034044, '_INCOMG1_$25,000 to < $35,000'),
 (0.01563137150101107, '_AGEG5YR_Age 60 to 64 '),
 (0.015597249291339765, '_AGEG5YR_Age 65 to 69 '),
 (0.015300314191977856, 'GENHLTH_Fair'),
 (0.015254549553825436, '_RACE_White'),
 (0.01430697220032041, '_AGEG5YR_Age 55 to 59 '),
 (0.014227573441810672, '_AGEG5YR_Age 70 to 74 '),
 (0.014041684754322227, 'TOLDHI3_Yes'),
 (0.013916326192317668, '_INCOMG1_$15,000 to < $25,000'),
 (0.013632826778061147, 'GENHLT